In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

import math

In [122]:
e = 1e-5

In [210]:
data = pd.read_csv('data-logistic.csv', header=None)
l = len(data.axes[0])
data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [211]:
def min_functional(data, w, C = 0):
    l = len(data.axes[0])
    
    sum = 0
    for i in range(0, l):
        y = data.ix[i][0]
        x1 = data.ix[i][1]
        x2 = data.ix[i][2]
        sum += math.log(1 + math.exp(-y * (w[0] * x1 + w[1] * x2)))
        
    w_mod = w[0]*w[0] + w[1]*w[1]
    sum += w_mod /2.0 *C
    return sum
    

In [232]:
def w_i_counting(j, data, w, k=0.1, C=0):
    l = len(data.axes[0])
    
    res = 0.0;
    for i in range(0, l):
        y = data.ix[i][0]
        x1 = data.ix[i][1]
        x2 = data.ix[i][2]
        x = (x1, x2)
        
        aa = w[0] * x1 + w[1] * x2
        aa = aa * (-y)
        res_iter = y * x[j] * (1 - 1.0 / (1 + math.exp( aa)))
        res += res_iter
    res = res * k / l
    
    res += w[j]
    res -= k * C * w[j]
    return res
    

In [213]:
def sigmoida(w, x):
    return 1.0 / (1 + math.exp( - w[0] * x[0] - w[1] * x[1] ))
    

In [214]:
w = (0.0, 0.0)
iter = 0
exit = 1
while iter < 1000 and exit == 1:
    w1 = w_i_counting(0, data, w)
    w2 = w_i_counting(1, data, w)
    if math.sqrt((w1 - w[0])*(w1 - w[0]) + (w2 - w[1])*(w2 - w[1])) < e:
        w = (w1, w2)
        exit = 0
    w = (w1, w2)
    iter += 1
    
w

(0.28781162047177639, 0.09198330215925439)

In [215]:
iter

244

In [222]:
y_predicted = []
for i in range(0, l):
    y_predicted.append(sigmoida(w, np.asarray(data.ix[i, 1:3])))
    
answer1 = roc_auc_score(data.ix[:, 0], y_predicted)
answer1

0.92685714285714282

In [233]:
w = (0.0, 0.0)
iter = 0
exit = 1
while iter < 1000 and exit == 1:
    w1 = w_i_counting(0, data, w, C=10)
    w2 = w_i_counting(1, data, w, C=10)
    if math.sqrt((w1 - w[0])*(w1 - w[0]) + (w2 - w[1])*(w2 - w[1])) < e:
        exit = 0
    w = (w1, w2)
    iter += 1
    
w

(0.028558754546234223, 0.024780137249735559)

In [234]:
iter

8

In [235]:
y_predicted = []
for i in range(0, l):
    y_predicted.append(sigmoida(w, np.asarray(data.ix[i, 1:3])))
    
answer2 = roc_auc_score(data.ix[:, 0], y_predicted)
answer2

0.93628571428571417

In [236]:
f = open('answers/3', 'w')
f.write('%.3f %.3f' % (answer1, answer2))
f.close()

!cat answers/3

0.927 0.936